# Analyzing product sentiment

In [26]:
import graphlab

In [27]:
products = graphlab.SFrame('amazon_baby.gl/')

### Build the word count vector for each review

In [28]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [29]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

### Use .apply() to build a new feature with the counts for each of the selected_words

In [30]:
selected_word_count = {}
for word in selected_words:
    products[word] = products['word_count'].apply(lambda dict: (word in dict))
    selected_word_count[word] = products[word].sum()
print sorted(selected_word_count.items(), key=lambda x: x[1])

[('wow', 132L), ('awful', 364L), ('horrible', 697L), ('terrible', 713L), ('fantastic', 908L), ('hate', 1153L), ('amazing', 1311L), ('awesome', 2017L), ('bad', 3518L), ('love', 34714L), ('great', 38038L)]


### Create a new sentiment analysis model using only the selected_words as features

In [31]:
#ignore all 3* reviews
products = products[products['rating'] != 3]
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >= 4
products.head()

name,review,rating,word_count,awesome,great,fantastic
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",0,0,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",0,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",0,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1L, 'forget': 1L,'just': 1L, ""daughter ...",0,0,0


In [32]:
train_data,test_data = products.random_split(.8, seed=0)

In [33]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 2         | 3        | 0.087723     | 0.844022          | 0.842662            |

| 3         | 4        | 0.121633     | 0.844201          | 0.843112            |

| 4         | 5        | 0.157537     | 0.844201          | 0.843112            |

| 5         | 6        | 0.189451     | 0.844201          | 0.843112            |

| 6         | 7        | 0.224358     | 0.844201          | 0.843112            |

In [34]:
selected_words_model['coefficients'].sort("value", ascending=False).print_rows(num_rows=12)

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

+-------------+-------+-------+-----------------+------------------+
|     name    | index | class |      value      |      stderr      |
+-------------+-------+-------+-----------------+------------------+
|     love    |  None |   1   |  1.64403341868  | 0.0317518510948  |
| (intercept) |  None |   1   |  1.35745751254  | 0.00865122839943 |
|   awesome   |  None |   1   |  1.10621291304  |  0.113671723976  |
|    great    |  None |   1   |  1.02735930291  | 0.0245599703114  |
|   amazing   |  None |   1   |  0.963966534307 |  0.133059786595  |
|  fantastic  |  None |   1   |  0.945038562814 |  0.159051764088  |
|     wow     |  None |   1   | -0.103737752058 |  0.278553086963  |
|     bad     |  None |   1   |  -1.07148926137 | 0.0461940387969  |
|     hate    |  None |   1   |  -1.45646982328 | 0.0801601689997  |
|    awful    |  None |   1   |  -1.81198950155 |  0.135501750222  |
|   horrible  |  None |   1   |  -2.09757202069 |  0.098900066701  |
|   terrible  |  None |   1   |  -

### Comparing the accuracy of different sentiment analysis model

In [35]:
# selected_words_model
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431119385058852,
 'auc': 0.6630846435243509,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  218  |
 |      1       |        0        |  115  |
 |      0       |        1        |  5110 |
 |      1       |        1        | 27861 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9142697753786075,
 'log_loss': 0.4058709564241642,
 'precision': 0.845015316490249,
 'recall': 0.9958893337146125,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 

In [36]:
# sentiment_model
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883,
 'auc': 0.9446492867438502,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  1461 |
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9500349343413533,
 'log_loss': 0.2610669843242172,
 'precision': 0.9523039902309378,
 'recall': 0.9477766657134686,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |  

In [37]:
# majority model
print test_data[test_data["sentiment"] == 1].num_rows() / float(test_data.num_rows())

0.840019216911


### Interpreting the difference in performance between the models

In [38]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [39]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

##Sort the reviews based on the predicted sentiment and explore

In [40]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [41]:
diaper_champ_reviews.head()

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1L, 'less': 1L,""friend's"": 1L, '(whi ...",0,0,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1L, 'over': 1L,'rweek': 1L, 'sooo': 1L, ...",0,0,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2L, 'just': 4L,""don't"": 2L, 'one,': 1L, ...",0,0,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1L, 'when': 1L,'over': 1L, 'soon': 1L, ...",0,1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3L, 'money': 1L,'not': 2L, 'mechanism': ...",0,0,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'all': 1L, 'bags.': 1L,'son,': 1L, '(i': 1L, ...",0,0,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4L, '""genie"".':1L, 'since': 1L, ...",0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1L, 'all': 2L,'bags.': 1L, 'feedback': ...",0,0,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",0,0,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'taller': 1L, 'bags.':1L, 'just': 1L, ""don't"": ...",0,0,0


In [42]:
# use the selected_words_model you learned using just the selected_words to predict the sentiment most positive review you found above
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.795346166148523]

In [43]:
diaper_champ_reviews[0]['review']

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'